# Day 20 
## Part 1
I'm going to start off doing what it says on the tin but I think it will need a cycle check.

In [1]:
from dataclasses import dataclass, field
from enum import IntEnum
from collections import defaultdict, deque
from pyrsistent import pmap


class Pulse(IntEnum):
    LOW = 0
    HIGH = 1


@dataclass
class FlipFlop:
    on: bool = False

    def receive(self, pulse, source):
        if not pulse:
            self.on = not self.on
            yield Pulse(self.on)


@dataclass
class Conjunction:
    received: defaultdict = field(default_factory=lambda: defaultdict(list))

    def receive(self, pulse, source):
        self.received[source] = pulse
        yield Pulse(not all(self.received.values()))


@dataclass 
class Broadcaster:
    def receive(self, pulse, source):
        yield pulse

In [2]:
def parse_data(s):
    modules = {}
    connections = defaultdict(list)
    
    for line in s.strip().splitlines():
        source, destinations = line.split(" -> ")
        match source[0]:
            case "%":
                label = source[1:]
                modules[label] = FlipFlop()
            case "&":
                label = source[1:]
                modules[label] = Conjunction()
            case _:
                label = source
                modules[label] = Broadcaster()
        connections[label] = destinations.split(", ")

    for m in modules:
        for c in connections[m]:
            if c in modules and isinstance(modules[c], Conjunction):
                modules[c].received[m] = Pulse.LOW

    return modules, connections

test_input = """broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output"""

parse_data(test_input)

({'broadcaster': Broadcaster(),
  'a': FlipFlop(on=False),
  'inv': Conjunction(received=defaultdict(<class 'list'>, {'a': <Pulse.LOW: 0>})),
  'b': FlipFlop(on=False),
  'con': Conjunction(received=defaultdict(<class 'list'>, {'a': <Pulse.LOW: 0>, 'b': <Pulse.LOW: 0>}))},
 defaultdict(list,
             {'broadcaster': ['a'],
              'a': ['inv', 'con'],
              'inv': ['b'],
              'b': ['con'],
              'con': ['output']}))

In [3]:
from itertools import count

def push_button(modules, connections):
    q = deque([("button", "broadcaster", Pulse.LOW)])
    counts = defaultdict(int)
    counts[Pulse.LOW] = 1

    while q:
        source, destination, pulse = q.popleft()
        if destination in modules:
            for p in modules[destination].receive(pulse, source):
                for c in connections[destination]:
                    q.append((destination, c, p))
                    counts[p] += 1

    return (modules, counts[Pulse.LOW], counts[Pulse.HIGH])

Tracking the cycles is going for a while so perhaps that won't work here, just do what's asked.

In [4]:
def part_1(s):
    modules, connections = parse_data(s)
    
    counts_low = 0
    counts_high = 0

    for _ in range(1000):
        modules, l, h = push_button(modules, connections)
        counts_low += l
        counts_high += h

    return counts_low * counts_high

part_1(test_input)

11687500

In [5]:
inp = open("input").read()
part_1(inp)

711650489

I'm not going to pretend that that didn't take some debugging, and the code's pretty horrible as a result.

## Part 2

This _looks_ simple.

In [6]:
def push_button(modules, connections):
    q = deque([("button", "broadcaster", Pulse.LOW)])

    while q:
        source, destination, pulse = q.popleft()
        found = False
        if destination in modules:
            for p in modules[destination].receive(pulse, source):
                for c in connections[destination]:
                    q.append((destination, c, p))
                    if destination == "rx" and p == pulse.LOW:
                        found = True

    return (modules, found)

def part_1(s):
    modules, connections = parse_data(s)
    
    counts_low = 0
    counts_high = 0

    for n in count(1):
        modules, found = push_button(modules, connections)
        if found:
            return n

# part_1(inp)

This is running very slowly. It looks like it needs decompiling.

In [7]:
modules, connections = parse_data(inp)

In [27]:
[c for c in connections if "rx" in connections[c]]

['dt']

In [28]:
modules["dt"]

Conjunction(received=defaultdict(<class 'list'>, {'ks': <Pulse.LOW: 0>, 'pm': <Pulse.LOW: 0>, 'dl': <Pulse.LOW: 0>, 'vk': <Pulse.LOW: 0>}))

So `dt` will output a low pulse when all four of `ks`, `pm`, `dl` and `vk` send high pulses. Before decompiling further let's see if there's a cycle there.

In [33]:
def push_button(modules, connections):
    q = deque([("button", "broadcaster", Pulse.LOW)])

    while q:
        source, destination, pulse = q.popleft()
        found = False
        if destination in modules:
            for p in modules[destination].receive(pulse, source):
                for c in connections[destination]:
                    q.append((destination, c, p))

    return modules

modules, connections = parse_data(inp)
# for i in count(1):
#     modules = push_button(modules, connections)
#     for m in ["ks", "pm", "dl", "vk"]:
#         if modules[m] == Pulse.HIGH:
#             print(m, i)
            

No, that's too slow too. Hmm.

In [34]:
modules["ks"]

Conjunction(received=defaultdict(<class 'list'>, {'vr': <Pulse.LOW: 0>}))

That will be high when `vr` sends a low pulse.

In [35]:
modules["vr"]

Conjunction(received=defaultdict(<class 'list'>, {'lz': <Pulse.LOW: 0>, 'gx': <Pulse.LOW: 0>, 'lt': <Pulse.LOW: 0>, 'tx': <Pulse.LOW: 0>, 'xz': <Pulse.LOW: 0>, 'sb': <Pulse.LOW: 0>, 'ng': <Pulse.LOW: 0>, 'cn': <Pulse.LOW: 0>}))

... which happens when all of these are high.

In [42]:
modules["lz"]

FlipFlop(on=False)

In [44]:
[c for c in connections if "lz" in connections[c]]

['sb']

In [46]:
modules["sb"]

FlipFlop(on=False)

In [47]:
[c for c in connections if "sb" in connections[c]]

['jg']

In [48]:
modules["jg"]

FlipFlop(on=False)

In [49]:
[c for c in connections if "jg" in connections[c]]

['tx', 'vr']

Ok, so there's a feedback loop from `vr`.

In [50]:
[c for c in connections if "tx" in connections[c]]

['vr', 'broadcaster']

And again here. And finally there's the broadcaster. I think I need to draw the whole network.

In [58]:
import networkx as nx
from pyvis.network import Network

G = nx.DiGraph()
for m in modules:
    if isinstance(modules[m], Conjunction):
        G.add_node(m, type="C", label=m, color="#ff0000")
    elif isinstance(modules[m], FlipFlop):
        G.add_node(m, type="F", label=m, color="#00ff00")
    else:
        G.add_node(m, type="B", label=m, color="#0000ff")
for c in connections:
    for n in connections[c]:
        G.add_edge(c, n)

nt = Network(notebook=True, directed=True)
nt.from_nx(G)
nt.show("circuit.html")

circuit.html
